### CONTENTS
#### 1 Importing Libraries
#### 2 Importing Data 
#### 3 Fitting Visual Weather Systems CNN

## 1 Importing Libraries

In [139]:
import os
import operator
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import Input, Model, Sequential
from tensorflow.keras.layers import (
    Conv1D, Conv2D, Dense, BatchNormalization, Flatten, 
    MaxPooling2D, Activation, Dropout
)
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import confusion_matrix, accuracy_score, ConfusionMatrixDisplay

In [141]:
import os, numpy as np, matplotlib.pyplot as plt
from sklearn.metrics import f1_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.utils.class_weight import compute_class_weight

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.applications.efficientnet import EfficientNetB0, preprocess_input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy

## 2 Importing Data

In [144]:
# Paths and constants
# -------------------
path = r'C:\Users\elena\Documents\06.2025 climatewins machine learning\Multi-class Weather Dataset'
img_size = (224, 224)
batch = 32
seed = 42


# 3 Fitting Visual Weather Systems CNN: EfficientNetB0 

In [146]:
# Generators (use preprocess_input; do NOT also rescale)
# -------------------
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2,
    rotation_range=20, width_shift_range=0.2, height_shift_range=0.2,
    zoom_range=0.2, horizontal_flip=True
)
val_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2
)

train_gen = train_datagen.flow_from_directory(
    path, target_size=img_size, batch_size=batch, subset='training',
    class_mode='categorical', shuffle=True, seed=seed
)
val_gen = val_datagen.flow_from_directory(
    path, target_size=img_size, batch_size=batch, subset='validation',
    class_mode='categorical', shuffle=False, seed=seed
)

class_indices = train_gen.class_indices
idx_to_cls = {v:k for k,v in class_indices.items()}
print("Class indices:", class_indices)

# -------------------
# Class weights (optional but helpful for imbalance)
# -------------------
cw = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_gen.classes),
    y=train_gen.classes
)
class_weights = {i:w for i,w in enumerate(cw)}
print("Class weights:", class_weights)

Found 901 images belonging to 4 classes.
Found 224 images belonging to 4 classes.
Class indices: {'Cloudy': 0, 'Rain': 1, 'Shine': 2, 'Sunrise': 3}
Class weights: {0: 0.9385416666666667, 1: 1.309593023255814, 2: 1.1096059113300492, 3: 0.7875874125874126}


In [148]:
# Build model
# -------------------
base = EfficientNetB0(weights="imagenet", include_top=False, input_shape=img_size + (3,))
base.trainable = False  # Stage 1: freeze

inputs = Input(shape=img_size + (3,))
x = base(inputs, training=False)             # BN layers stay in inference for stage 1
x = GlobalAveragePooling2D()(x)
x = Dropout(0.4)(x)
outputs = Dense(train_gen.num_classes, activation='softmax')(x)
model = Model(inputs, outputs)


In [150]:
# Compile (stage 1)
# -------------------
model.compile(
    optimizer=Adam(learning_rate=1e-3),
    loss=CategoricalCrossentropy(label_smoothing=0.05),  # small smoothing helps generalization
    metrics=['accuracy']
)

es = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
rlrop = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)
ckpt = ModelCheckpoint("effb0_best.h5", monitor='val_loss', save_best_only=True, verbose=1)

history = model.fit(
    train_gen,
    epochs=20,
    validation_data=val_gen,
    class_weight=class_weights,
    callbacks=[es, rlrop, ckpt],
    verbose=1
)


C:\Users\elena\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.4208 - loss: 1.3128   
Epoch 1: val_loss improved from inf to 0.75551, saving model to effb0_best.h5


29/29 ━━━━━━━━━━━━━━━━━━━━ 54s 2s/step - accuracy: 0.4259 - loss: 1.3054 - val_accuracy: 0.7946 - val_loss: 0.7555 - learning_rate: 0.0010
Epoch 2/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.8182 - loss: 0.6892 
Epoch 2: val_loss improved from 0.75551 to 0.58789, saving model to effb0_best.h5


29/29 ━━━━━━━━━━━━━━━━━━━━ 39s 1s/step - accuracy: 0.8195 - loss: 0.6870 - val_accuracy: 0.8438 - val_loss: 0.5879 - learning_rate: 0.0010
Epoch 3/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.8926 - loss: 0.5000 
Epoch 3: val_loss improved from 0.58789 to 0.51502, saving model to effb0_best.h5


29/29 ━━━━━━━━━━━━━━━━━━━━ 39s 1s/step - accuracy: 0.8925 - loss: 0.5000 - val_accuracy: 0.8571 - val_loss: 0.5150 - learning_rate: 0.0010
Epoch 4/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9277 - loss: 0.4419 
Epoch 4: val_loss improved from 0.51502 to 0.47751, saving model to effb0_best.h5


...

29/29 ━━━━━━━━━━━━━━━━━━━━ 41s 1s/step - accuracy: 0.9610 - loss: 0.3219 - val_accuracy: 0.9152 - val_loss: 0.3844 - learning_rate: 0.0010
Epoch 16/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9765 - loss: 0.3093 
Epoch 16: val_loss improved from 0.38445 to 0.38236, saving model to effb0_best.h5


29/29 ━━━━━━━━━━━━━━━━━━━━ 41s 1s/step - accuracy: 0.9762 - loss: 0.3096 - val_accuracy: 0.9152 - val_loss: 0.3824 - learning_rate: 0.0010
Epoch 17/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9631 - loss: 0.3330 
Epoch 17: val_loss improved from 0.38236 to 0.37991, saving model to effb0_best.h5


29/29 ━━━━━━━━━━━━━━━━━━━━ 40s 1s/step - accuracy: 0.9631 - loss: 0.3329 - val_accuracy: 0.9152 - val_loss: 0.3799 - learning_rate: 0.0010
Epoch 18/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9752 - loss: 0.3129 
Epoch 18: val_loss improved from 0.37991 to 0.37692, saving model to effb0_best.h5


29/29 ━━━━━━━━━━━━━━━━━━━━ 40s 1s/step - accuracy: 0.9749 - loss: 0.3130 - val_accuracy: 0.9241 - val_loss: 0.3769 - learning_rate: 0.0010
Epoch 19/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9672 - loss: 0.3157 
Epoch 19: val_loss improved from 0.37692 to 0.37359, saving model to effb0_best.h5


29/29 ━━━━━━━━━━━━━━━━━━━━ 39s 1s/step - accuracy: 0.9671 - loss: 0.3159 - val_accuracy: 0.9241 - val_loss: 0.3736 - learning_rate: 0.0010
Epoch 20/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9735 - loss: 0.3031    
Epoch 20: val_loss did not improve from 0.37359
29/29 ━━━━━━━━━━━━━━━━━━━━ 39s 1s/step - accuracy: 0.9733 - loss: 0.3033 - val_accuracy: 0.9196 - val_loss: 0.3739 - learning_rate: 0.0010


In [152]:
# Fine-tune (stage 2)
# -------------------
base.trainable = True
# Unfreeze only the top block(s) to start; adjust as needed
for layer in base.layers[:-30]:
    layer.trainable = False

model.compile(
    optimizer=Adam(learning_rate=1e-5),  # MUCH lower LR for fine-tuning
    loss=CategoricalCrossentropy(label_smoothing=0.05),
    metrics=['accuracy']
)

history_ft = model.fit(
    train_gen,
    epochs=20,
    validation_data=val_gen,
    class_weight=class_weights,
    callbacks=[es, rlrop, ckpt],
    verbose=1
)

Epoch 1/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.8332 - loss: 0.6426   
Epoch 1: val_loss did not improve from 0.37359
29/29 ━━━━━━━━━━━━━━━━━━━━ 57s 2s/step - accuracy: 0.8335 - loss: 0.6413 - val_accuracy: 0.9241 - val_loss: 0.3856 - learning_rate: 1.0000e-05
Epoch 2/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.8618 - loss: 0.5828 
Epoch 2: val_loss did not improve from 0.37359
29/29 ━━━━━━━━━━━━━━━━━━━━ 43s 1s/step - accuracy: 0.8626 - loss: 0.5816 - val_accuracy: 0.9196 - val_loss: 0.4016 - learning_rate: 1.0000e-05
Epoch 3/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.8991 - loss: 0.4984 
Epoch 3: val_loss did not improve from 0.37359
29/29 ━━━━━━━━━━━━━━━━━━━━ 44s 2s/step - accuracy: 0.8991 - loss: 0.4989 - val_accuracy: 0.9107 - val_loss: 0.4192 - learning_rate: 1.0000e-05
Epoch 4/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9113 - loss: 0.4913    
Epoch 4: val_loss did not improve from 0.37359
29/29 ━━━━━━━━━━━━━━━━━━━━ 44s 2s/step - a

In [154]:
# Evaluate (weighted F1 + confusion matrix)
# -------------------
val_gen.reset()
y_prob = model.predict(val_gen)
y_pred = np.argmax(y_prob, axis=1)
y_true = val_gen.classes

weighted_f1 = f1_score(y_true, y_pred, average='weighted')
print("\nWeighted F1:", weighted_f1)

target_names = [idx_to_cls[i] for i in range(len(idx_to_cls))]
print("\nClassification Report:\n", classification_report(y_true, y_pred, target_names=target_names))

cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(cm, display_labels=target_names)
disp.plot(cmap='Blues', values_format='d')
plt.title("Confusion Matrix - EfficientNetB0 (proper preprocessing + low-LR fine-tune)")
plt.tight_layout()
plt.show()

7/7 ━━━━━━━━━━━━━━━━━━━━ 9s 967ms/step

Weighted F1: 0.9239845809264524

Classification Report:
               precision    recall  f1-score   support

      Cloudy       0.86      0.95      0.90        60
        Rain       0.98      1.00      0.99        43
       Shine       0.87      0.82      0.85        50
     Sunrise       0.99      0.93      0.96        71

    accuracy                           0.92       224
   macro avg       0.92      0.92      0.92       224
weighted avg       0.93      0.92      0.92       224



This modification to the original model significantly improved model performance, as demonstraited by weighter F1 of 0.92 and the improved Confusion Matrix